In [1]:
import numpy as np

from src.test_data import case_list, y
from src.model import *
from src.similarity import sim_attr, get_alternative
from src.spa import *
from src.utils import *

In [ ]:
alpha = 0.5
tao = 0.7

alternative_list = get_alternative(case_list, y, 0.5, 0.7)

em_plan = set_pari_analysis(alternative_list)

In [3]:
em_plan.scenario.get_attrvalues()

['北部林区', 3, 3, [4, 18], 5.5, nan, 3600, nan, 5]

In [4]:
em_plan.effect.get_attrvalues()

[1173, 202.5, 522, 0.4, 0.95, 0.65]

# 基于遗传算法修正属性值

## 交叉

In [ ]:
action_list = get_knowledge(alternative_list, 'action')

# 共有属性的相似度
similarity_attr = sim_attr(action_list, y.action) # (m, n) m个行动，n个属性

# 每种相似度最高的属性索引
indices = np.argmax(similarity_attr, axis=0)

# 共有属性值矩阵
common_attr = y.action.get_common_attrs(action_list)
action_matrix = attr_matrix(common_attr, action_list)

# 取出相似度最高的属性值
best_attr = action_matrix[indices, np.arange(len(common_attr))]

# 修改初步应急方案中的属性值
for attr_name, value in zip(common_attr, best_attr):
    em_plan.action.update_attribute(attr_name, value)

## 变异

In [ ]:
for case in alternative_list:
    # 找到备选方案集中未参与交叉的属性
    sup_attr = set(case.action.attr_names) - set(case.action.get_common_attrs([em_plan.action]))
    
    sup_value = case.action.get_attrvalues()
    sup_dtype = case.action.get_attr_dtype(sup_attr)
    sup_unit = case.action.get_unit(sup_attr)
    
    attr_list = []
    
    for i in zip(sup_attr, sup_value, sup_dtype, sup_unit):
        attr_list.append(list(i))
    em_plan.action.add_attributes(attr_list)